In [1]:
# Import Library


# Model
import numpy as np
import tensorflow as tf
import random

# Kontekstualisasi dan Pekerjaan NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Pendukung
import json
import pickle
import warnings
import curses

warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
print(tf.__version__)

2.12.0


In [4]:
# Upload Dataset
print("Processing the intents")
with open('sample_data/intents.json') as file:
  data = json.load(file)

Processing the intents


In [5]:
print(data)

{'intents': [{'tag': 'salam', 'patterns': ['Hai', 'Halo', 'Assalamualaikum', 'Hei', 'Selamat Pagi', 'Selamat Siang', 'Selamat Sore', 'Selamat Malam', 'Pagi', 'Siang', 'Sore', 'Malam', 'Malem', 'Hallo Admin', 'Halo Admin', 'Hai Admin'], 'responses': ['Halo, Selamat datang di JasaIn Chatbot!. Jika kamu mempunyai kendala, silahkan deskripsikan kendalamu ya!']}, {'tag': 'penutup', 'patterns': ['Terimakasih', 'Makasih', 'Thank You', 'Makasih Banyak', 'Terimakasih Banyak', 'Terimaksih atas Bantuannya', 'Makasih atas Bantuannya', 'Oke Makasih', 'Oke Terimakasih', 'Oke terimakasih atas bantuannya'], 'responses': ['Sama-sama, Senang Bisa membantu!, Terimakasih telah menghubungi kami']}, {'tag': 'hp_lcd', 'patterns': ['Saya punya kendala, LCD hp saya kadang-kadang tidak bisa dipencet', 'Hp saya tidak bisa ditekan', 'Layar saya tidak bisa dipencet', 'layar Hp sya mati tetapi masih ada suara notifikasi', 'Layar Hp saya tidak bisa disentuh', 'Layar Hp gak bisa disentuh', 'Touchscreen hp saya tidak 

In [6]:
# Mengelompokkan kata
words = []
classes = []
documents = []
ignore_words = []
print('"Ulangi Niat untuk Mengonversinya menjadi kata, kelas, dokumen, dan abaikan kata...."')
for intent in data['intents']:
  for pattern in intent['patterns']:
    # Tokenisasi tiap kata dalam kalimat
    w = nltk.word_tokenize(pattern)
    # Menambahkan hasil tokenisasi ke dalam array words
    words.extend(w)
    # Menambahkan dokumen ke dalam korpus(pelatihan/uji)
    documents.append((w, intent['tag']))
    # Menambahkan tag kedalama array class
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

"Ulangi Niat untuk Mengonversinya menjadi kata, kelas, dokumen, dan abaikan kata...."


In [7]:
print(np.array(words))

['Hai' 'Halo' 'Assalamualaikum' 'Hei' 'Selamat' 'Pagi' 'Selamat' 'Siang'
 'Selamat' 'Sore' 'Selamat' 'Malam' 'Pagi' 'Siang' 'Sore' 'Malam' 'Malem'
 'Hallo' 'Admin' 'Halo' 'Admin' 'Hai' 'Admin' 'Terimakasih' 'Makasih'
 'Thank' 'You' 'Makasih' 'Banyak' 'Terimakasih' 'Banyak' 'Terimaksih'
 'atas' 'Bantuannya' 'Makasih' 'atas' 'Bantuannya' 'Oke' 'Makasih' 'Oke'
 'Terimakasih' 'Oke' 'terimakasih' 'atas' 'bantuannya' 'Saya' 'punya'
 'kendala' ',' 'LCD' 'hp' 'saya' 'kadang-kadang' 'tidak' 'bisa' 'dipencet'
 'Hp' 'saya' 'tidak' 'bisa' 'ditekan' 'Layar' 'saya' 'tidak' 'bisa'
 'dipencet' 'layar' 'Hp' 'sya' 'mati' 'tetapi' 'masih' 'ada' 'suara'
 'notifikasi' 'Layar' 'Hp' 'saya' 'tidak' 'bisa' 'disentuh' 'Layar' 'Hp'
 'gak' 'bisa' 'disentuh' 'Touchscreen' 'hp' 'saya' 'tidak' 'berfungsi'
 'Hp' 'saya' 'tidak' 'bisa' 'di' 'cas' 'Hp' 'saya' 'tidak' 'bisa' 'di'
 'charger' 'Kenapa' 'ya' 'hp' 'saya' 'tidak' 'bisa' 'di' 'cas' 'Kenapa'
 'ya' 'hp' 'saya' 'tidak' 'bisa' 'di' 'charger' 'Saya' 'sudah' 'cas' 'h

In [8]:
np.array(classes)

array(['salam', 'penutup', 'hp_lcd', 'hp_iccharger', 'hp_bootlop',
       'hp_baterai', 'hp_overheat', 'hp_wifi', 'hp_bluetooth', 'ac_arti',
       'ac_merek', 'ac_rusak', 'ac_kotor', 'ac_freon', 'ac_kompresor',
       'ac_bocor', 'create_account', 'complaint', 'laptop_overheating',
       'laptop_keyboard_not_working', 'laptop_blue_screen',
       'laptop_battery_issues', 'laptop_screen_problems',
       'laptop_wifi_issues', 'laptop_hard_drive_failure',
       'laptop_usb_port_issues', 'laptop_audio_problems',
       'laptop_slow_performance', 'laptop_virus_infection',
       'laptop_screen_flickering', 'laptop_charging_issues',
       'laptop_driver_errors', 'laptop_startup_issues'], dtype='<U27')

In [9]:
np.array(documents)

array([[list(['Hai']), 'salam'],
       [list(['Halo']), 'salam'],
       [list(['Assalamualaikum']), 'salam'],
       [list(['Hei']), 'salam'],
       [list(['Selamat', 'Pagi']), 'salam'],
       [list(['Selamat', 'Siang']), 'salam'],
       [list(['Selamat', 'Sore']), 'salam'],
       [list(['Selamat', 'Malam']), 'salam'],
       [list(['Pagi']), 'salam'],
       [list(['Siang']), 'salam'],
       [list(['Sore']), 'salam'],
       [list(['Malam']), 'salam'],
       [list(['Malem']), 'salam'],
       [list(['Hallo', 'Admin']), 'salam'],
       [list(['Halo', 'Admin']), 'salam'],
       [list(['Hai', 'Admin']), 'salam'],
       [list(['Terimakasih']), 'penutup'],
       [list(['Makasih']), 'penutup'],
       [list(['Thank', 'You']), 'penutup'],
       [list(['Makasih', 'Banyak']), 'penutup'],
       [list(['Terimakasih', 'Banyak']), 'penutup'],
       [list(['Terimaksih', 'atas', 'Bantuannya']), 'penutup'],
       [list(['Makasih', 'atas', 'Bantuannya']), 'penutup'],
       [list(['Oke

In [10]:
np.array(ignore_words)

array([], dtype=float64)

In [11]:
print('Stemming, Downgrade dan Menghapus Duplikat')
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Menghapus Duplikat
classes = sorted(list(set(classes)))

print(len(documents), "documents")

Stemming, Downgrade dan Menghapus Duplikat
217 documents


In [12]:
print(len(classes), 'classes')
print(np.array(classes))

33 classes
['ac_arti' 'ac_bocor' 'ac_freon' 'ac_kompresor' 'ac_kotor' 'ac_merek'
 'ac_rusak' 'complaint' 'create_account' 'hp_baterai' 'hp_bluetooth'
 'hp_bootlop' 'hp_iccharger' 'hp_lcd' 'hp_overheat' 'hp_wifi'
 'laptop_audio_problems' 'laptop_battery_issues' 'laptop_blue_screen'
 'laptop_charging_issues' 'laptop_driver_errors'
 'laptop_hard_drive_failure' 'laptop_keyboard_not_working'
 'laptop_overheating' 'laptop_screen_flickering' 'laptop_screen_problems'
 'laptop_slow_performance' 'laptop_startup_issues'
 'laptop_usb_port_issues' 'laptop_virus_infection' 'laptop_wifi_issues'
 'penutup' 'salam']


In [13]:
print(len(words), 'kata unik')
print(words)

215 kata unik
[',', '?', 'ac', 'ad', 'adalah', 'admin', 'air', 'ak', 'aku', 'akun', 'ap', 'artiny', 'assalamualaik', 'ata', 'bag', 'bagaiman', 'bantuanny', 'banyak', 'baru', 'bat', 'batera', 'baterainy', 'bau', 'berair', 'berfungs', 'berkedip', 'berkedip-kedip', 'berkurang', 'bermasalah', 'bersih', 'bertah', 'bertambah', 'biru', 'bis', 'bisakah', 'blank', 'blu', 'bluetoo', 'bluetoothny', 'blutut', 'boc', 'butuh', 'car', 'cas', 'casanny', 'charg', 'cum', 'dan', 'dap', 'dar', 'day', 'definis', 'di', 'dibersihk', 'dica', 'diis', 'dinyalain', 'dinyalak', 'dipencet', 'diperbaik', 'disentuh', 'diskny', 'ditek', 'driv', 'engg', 'er', 'freon', 'ga', 'gabis', 'gak', 'gamb', 'gampang', 'gant', 'gk', 'gua', 'gue', 'hab', 'hai', 'hallo', 'halo', 'handphon', 'hany', 'hard', 'hdd', 'hei', 'hp', 'hpku', 'in', 'ingin', 'itu', 'jatuh', 'kadang-kadang', 'kalau', 'kamu', 'ke', 'keluh', 'kenak', 'kenap', 'kendal', 'kepanjanganny', 'kerusak', 'kesalah', 'keyboard', 'keyboardny', 'kinerj', 'kompres', 'kompr

In [14]:
print('Membuat data untuk model kita')
training = []
output = []

print('Membuat sebuah list kosong untuk output')
output_empty = [0] * len(classes)
print(output_empty)

Membuat data untuk model kita
Membuat sebuah list kosong untuk output
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
print('Membuat Training Set, Bag of Words untuk Model kita')
for doc in documents:
  # Inisialisasi Bow kita
  bag = []
  # daftar kata-kata yang diberi token untuk pola tersebut
  pattern_words = doc[0]
  # stem tiap kata
  pattern_words = [stemmer.stem(w.lower()) for w in pattern_words]
  # membuat BoW array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output adalah '0' untuk setiap tag dan '1' untuk tag saat ini
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Membuat Training Set, Bag of Words untuk Model kita


In [16]:
print(np.array(bag))
print()
print(np.array(output_row))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [17]:
# Cetak training dan BoW
print(np.array(training))

[[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
print('Mengacak Secara Acak dan Mengonversi ke Numpy Array untuk Pemrosesan Lebih Cepat')
random.shuffle(training)
training = np.array(training)

Mengacak Secara Acak dan Mengonversi ke Numpy Array untuk Pemrosesan Lebih Cepat


In [19]:
print(training)

[[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
print('Membuat train dan test list')
train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

Membuat train dan test list


In [21]:
print('Ukuran train_x', np.shape(train_x))
print('Ukuran train_y', np.shape(train_y))

Ukuran train_x (46655, 215)
Ukuran train_y (46655, 33)


In [22]:
# Membuat model
from tensorflow.keras import layers
from tensorflow.keras.initializers import TruncatedNormal
initializer = TruncatedNormal()

model = tf.keras.models.Sequential()
model.add(layers.Dense(8, input_shape=(len(train_x[0]),), activation='relu', kernel_initializer=initializer))
model.add(layers.Dense(len(train_y[0]), activation='softmax', kernel_initializer=initializer))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 1728      
                                                                 
 dense_1 (Dense)             (None, 33)                297       
                                                                 
Total params: 2,025
Trainable params: 2,025
Non-trainable params: 0
_________________________________________________________________


In [31]:
print(len(train_y[0]))

33


In [30]:
print('Training model')
# Pastikan bentuk masukan cocok dengan jumlah fitur masukan

model.fit(train_x, train_y, batch_size=8, epochs=3)

Training model
Epoch 1/3
5832/5832 [==============================] - 12s 2ms/step - loss: 7.6649e-08 - acc: 1.0000
Epoch 2/3
5832/5832 [==============================] - 12s 2ms/step - loss: 2.7077e-08 - acc: 1.0000
Epoch 3/3
5832/5832 [==============================] - 11s 2ms/step - loss: 2.5771e-08 - acc: 1.0000


In [24]:
print('Simpan Model')
model.save('model.h5')

Simpan Model


In [25]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [26]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('sample_data/intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model = tf.keras.models.load_model('./model.h5')

Loading Pickle.....
Loading the Model......


In [27]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag).reshape(1, -1)  # Reshape to (1, num_features)


ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [28]:
while True:
    input_data = input("You- ")
    if input_data.lower() in ['exit', 'selamat tinggal']:
        break
    else:
        answer = response(input_data)

You- halo
1/1 [==============================] - 0s 123ms/step
Halo, Selamat datang di JasaIn Chatbot!. Jika kamu mempunyai kendala, silahkan deskripsikan kendalamu ya!
You- laptop saya rusak min
1/1 [==============================] - 0s 21ms/step
Silakan sebutkan keluhanmu, kami akan coba membantumu
You- ac saya rusak min
1/1 [==============================] - 0s 24ms/step
Mohon berikan keluhanmu agar kami bisa membantumu
You- laptop saya suka overheat 
1/1 [==============================] - 0s 21ms/step
Masalah overheating pada laptop bisa disebabkan oleh kipas yang kotor atau pendingin yang tidak efektif. Pastikan laptop Anda memiliki ventilasi yang baik dan coba bersihkan kipas dari debu secara berkala.


KeyboardInterrupt: ignored

In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)